In [ ]:
# ==========================================================
# TMDB TV SERIES + SEASONS HYPER-OPTIMIZED ASYNC COLLECTOR
# 비동기 방식 + 병목 제거 + 캐싱
# ✨ 최종 개선: providers를 flatrate/rent/buy로 분리 + imdb_id 추가
# ==========================================================

import os
import asyncio
import aiohttp
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
import time
from functools import lru_cache
import json

# ==========================================================
# Jupyter async 지원
# ==========================================================
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("✅ nest_asyncio 적용 완료")
except:
    pass

# ==========================================================
# 설정
# ==========================================================
load_dotenv()
API_KEY = os.getenv("API_KEY")
if not API_KEY:
    raise ValueError("API_KEY missing")

BASE_URL = "https://api.themoviedb.org/3"
HEADERS = {"accept": "application/json"}

# 🔥 핵심: TMDB 실제 제한은 50/초이지만, 버스트를 고려해 40으로 안정화
MAX_CALLS_PER_SECOND = 25
TIMEOUT = aiohttp.ClientTimeout(total=15, connect=5, sock_read=10)
MAX_RETRIES = 2

START_DATE = "2005-01-01"
END_DATE = "2015-12-31"

SERIES_TEMP = Path("tv_series_temp.csv")
SEASONS_TEMP = Path("tv_seasons_temp.csv")

SERIES_CSV = "tv_series_2005_2015_FULL.csv"
SEASONS_CSV = "tv_seasons_2005_2015_FULL.csv"
SERIES_PARQ = "tv_series_2005_2015_FULL.parquet"
SEASONS_PARQ = "tv_seasons_2005_2015_FULL.parquet"

# 통계
stats = {"requests": 0, "errors": 0, "retries": 0, "start_time": None}

# ==========================================================
# 고성능 Rate Limiter (Lock-free 버전)
# ==========================================================
class LockFreeRateLimiter:
    """Lock 없이 동작하는 초고속 Rate Limiter"""
    def __init__(self, rate):
        self.rate = rate
        self.tokens = rate
        self.updated_at = time.monotonic()
    
    async def acquire(self):
        now = time.monotonic()
        elapsed = now - self.updated_at
        
        # 토큰 보충 (Lock 없이)
        self.tokens = min(self.rate, self.tokens + elapsed * self.rate)
        self.updated_at = now
        
        # 토큰 부족시 최소 대기
        if self.tokens < 1:
            sleep_time = (1 - self.tokens) / self.rate
            await asyncio.sleep(sleep_time)
            self.tokens = 1
        
        self.tokens -= 1

rate_limiter = LockFreeRateLimiter(MAX_CALLS_PER_SECOND)

# ==========================================================
# TMDB GET (재시도 로직 개선)
# ==========================================================
async def tmdb_get(session, path, params=None, retry=0):
    if retry >= MAX_RETRIES:
        stats["errors"] += 1
        return None

    params = params or {}
    params.setdefault("api_key", API_KEY)
    params.setdefault("language", "en-US")

    url = f"{BASE_URL}{path}"
    
    await rate_limiter.acquire()
    stats["requests"] += 1

    try:
        async with session.get(url, params=params, headers=HEADERS) as resp:
            # 429: Rate limit
            if resp.status == 429:
                stats["retries"] += 1
                retry_after = int(resp.headers.get("Retry-After", 2))
                await asyncio.sleep(retry_after)
                return await tmdb_get(session, path, params, retry + 1)
            
            # 404: Not found (정상 케이스)
            if resp.status == 404:
                return None
            
            # 5xx: 서버 에러
            if 500 <= resp.status < 600:
                stats["retries"] += 1
                await asyncio.sleep(0.5 ** retry)  # 지수 감소
                return await tmdb_get(session, path, params, retry + 1)
            
            resp.raise_for_status()
            return await resp.json()
    
    except asyncio.TimeoutError:
        stats["errors"] += 1
        if retry < MAX_RETRIES - 1:
            await asyncio.sleep(0.2)
            return await tmdb_get(session, path, params, retry + 1)
        return None
    
    except Exception as e:
        stats["errors"] += 1
        if retry < MAX_RETRIES - 1:
            await asyncio.sleep(0.1)
            return await tmdb_get(session, path, params, retry + 1)
        return None

# ==========================================================
# Discover
# ==========================================================
async def fetch_discover_page(session, page, gte, lte):
    params = {
        "sort_by": "popularity.desc",
        "first_air_date.gte": gte,
        "first_air_date.lte": lte,
        "page": page,
        "include_adult": "true",
    }
    data = await tmdb_get(session, "/discover/tv", params)
    if not data:
        return [], 1, 0
    return data.get("results", []), data.get("total_pages", 1), data.get("total_results", 0)

# ==========================================================
# ID 수집 (개선)
# ==========================================================
async def collect_ids_in_range(session, start, end, depth=0):
    res1, total_pages, total_results = await fetch_discover_page(session, 1, start, end)

    if total_results == 0:
        return set()

    if total_pages <= 500:
        ids = {r["id"] for r in res1}
        
        if total_pages > 1:
            # 🔥 개선: 청크로 나눠서 메모리 효율 개선
            chunk_size = 100
            for chunk_start in range(2, total_pages + 1, chunk_size):
                chunk_end = min(chunk_start + chunk_size, total_pages + 1)
                results = await asyncio.gather(
                    *[fetch_discover_page(session, p, start, end) for p in range(chunk_start, chunk_end)],
                    return_exceptions=True
                )
                for r in results:
                    if isinstance(r, tuple):
                        ids.update(x["id"] for x in r[0])
        
        return ids

    # 날짜 분할
    start_dt = datetime.strptime(start, "%Y-%m-%d")
    end_dt = datetime.strptime(end, "%Y-%m-%d")
    mid_dt = start_dt + (end_dt - start_dt) / 2
    mid = mid_dt.strftime("%Y-%m-%d")
    right_start = (mid_dt + timedelta(days=1)).strftime("%Y-%m-%d")

    left, right = await asyncio.gather(
        collect_ids_in_range(session, start, mid, depth + 1),
        collect_ids_in_range(session, right_start, end, depth + 1)
    )
    return left | right

# ==========================================================
# Helpers (캐싱 추가)
# ==========================================================
@lru_cache(maxsize=1024)
def list_to_str_cached(lst_tuple, key="name"):
    """리스트를 문자열로 변환 (캐싱)"""
    if not lst_tuple:
        return ""
    return ", ".join(str(i.get(key, "")) for i in lst_tuple if i.get(key))

def list_to_str(lst, key="name"):
    if not lst:
        return ""
    # 딕셔너리 리스트를 튜플로 변환하여 캐싱 가능하게
    try:
        lst_tuple = tuple(tuple(d.items()) if isinstance(d, dict) else d for d in lst)
        return list_to_str_cached(lst_tuple, key)
    except:
        return ", ".join(str(i.get(key, "")) for i in lst if i.get(key))

def list_ids_to_str(lst, key="id"):
    if not lst:
        return ""
    return ", ".join(str(i.get(key)) for i in lst if i.get(key))

# ==========================================================
# ✨ NEW: External IDs (IMDB ID)
# ==========================================================
async def fetch_external_ids(session, sid):
    """시리즈의 external IDs를 가져와서 IMDB ID 반환"""
    data = await tmdb_get(session, f"/tv/{sid}/external_ids")
    if not data:
        return ""
    return data.get("imdb_id", "")

# ==========================================================
# ✨ UPDATED: Providers (flatrate/rent/buy 분리)
# ==========================================================
async def fetch_providers(session, sid):
    """
    모든 국가의 streaming providers 수집
    flatrate(구독형), rent(대여), buy(구매) 타입별로 분리
    반환: (flatrate_dict, rent_dict, buy_dict)
    """
    data = await tmdb_get(session, f"/tv/{sid}/watch/providers")
    if not data:
        return "{}", "{}", "{}"
    
    providers_results = data.get("results", {})
    
    # 국가별 OTT 정보 수집
    providers_flatrate = {}
    providers_rent = {}
    providers_buy = {}
    
    for country, info in providers_results.items():
        # flatrate: 구독형 (Netflix, Disney+ 등)
        flatrate = info.get("flatrate", [])
        if flatrate:
            provider_names = [p.get("provider_name") for p in flatrate if p.get("provider_name")]
            if provider_names:
                providers_flatrate[country] = provider_names
        
        # rent: 대여
        rent = info.get("rent", [])
        if rent:
            provider_names = [p.get("provider_name") for p in rent if p.get("provider_name")]
            if provider_names:
                providers_rent[country] = provider_names
        
        # buy: 구매
        buy = info.get("buy", [])
        if buy:
            provider_names = [p.get("provider_name") for p in buy if p.get("provider_name")]
            if provider_names:
                providers_buy[country] = provider_names
    
    # JSON 문자열로 변환하여 반환
    return (
        json.dumps(providers_flatrate, ensure_ascii=False) if providers_flatrate else "{}",
        json.dumps(providers_rent, ensure_ascii=False) if providers_rent else "{}",
        json.dumps(providers_buy, ensure_ascii=False) if providers_buy else "{}"
    )

# ==========================================================
# Single Season
# ==========================================================
async def fetch_single_season(session, sid, sname, net_names, net_ids, meta):
    sn = meta.get("season_number")
    if sn is None:
        return None

    data = await tmdb_get(session, f"/tv/{sid}/season/{sn}")

    if not data:
        return {
            "_id": f"{sid}_{sn}",
            "season_id": meta.get("id"),
            "series_id": sid,
            "series_name": sname,
            "season_number": sn,
            "name": meta.get("name"),
            "air_date": meta.get("air_date"),
            "overview": meta.get("overview"),
            "vote_average": meta.get("vote_average"),
            "vote_count": meta.get("vote_count"),
            "network_names": net_names,
            "network_ids": net_ids,
            "total_episodes": meta.get("episode_count"),
            "avg_episode_runtime": None,
            "poster_path": meta.get("poster_path"),
        }

    eps = data.get("episodes") or []
    runtimes = [ep["runtime"] for ep in eps if ep.get("runtime")]
    avg_rt = sum(runtimes) / len(runtimes) if runtimes else None

    return {
        "_id": data.get("_id") or f"{sid}_{sn}",
        "season_id": data.get("id"),
        "series_id": sid,
        "series_name": sname,
        "season_number": data.get("season_number"),
        "name": data.get("name"),
        "air_date": data.get("air_date"),
        "overview": data.get("overview"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
        "network_names": net_names,
        "network_ids": net_ids,
        "total_episodes": len(eps),
        "avg_episode_runtime": avg_rt,
        "poster_path": data.get("poster_path"),
    }

# ==========================================================
# ✨ UPDATED: TV Details + Seasons + IMDB ID + Providers
# ==========================================================
async def fetch_tv_details_and_seasons(session, sid):
    params = {"append_to_response": "reviews,keywords,aggregate_credits"}
    data = await tmdb_get(session, f"/tv/{sid}", params)
    if not data:
        return None, []

    genres = data.get("genres") or []
    networks = data.get("networks") or []
    last_ep = data.get("last_episode_to_air") or {}

    net_names = list_to_str(networks)
    net_ids = list_ids_to_str(networks)

    series = {
        "id": data.get("id"),
        "title": data.get("name"),
        "type": "tv_series",
        "adult": data.get("adult"),
        "backdrop_path": data.get("backdrop_path"),
        "created_by": list_to_str(data.get("created_by")),
        "episode_run_time": ", ".join(map(str, data.get("episode_run_time", []))) or "",
        "first_air_date": data.get("first_air_date"),
        "genres": list_to_str(genres),
        "genre_ids": ", ".join(str(g["id"]) for g in genres if g.get("id")),
        "homepage": data.get("homepage"),
        "in_production": data.get("in_production"),
        "languages": ", ".join(data.get("languages", [])) or "",
        "last_air_date": data.get("last_air_date"),
        "last_episode_to_air_id": last_ep.get("id"),
        "last_episode_to_air_name": last_ep.get("name"),
        "last_episode_to_air_overview": last_ep.get("overview"),
        "last_episode_to_air_vote_average": last_ep.get("vote_average"),
        "last_episode_to_air_vote_count": last_ep.get("vote_count"),
        "last_episode_to_air_air_date": last_ep.get("air_date"),
        "last_episode_to_air_episode_number": last_ep.get("episode_number"),
        "last_episode_to_air_production_code": last_ep.get("production_code"),
        "last_episode_to_air_runtime": last_ep.get("runtime"),
        "last_episode_to_air_season_number": last_ep.get("season_number"),
        "last_episode_to_air_show_id": last_ep.get("show_id"),
        "last_episode_to_air_still_path": last_ep.get("still_path"),
        "next_episode_to_air": str(data.get("next_episode_to_air")) if data.get("next_episode_to_air") else "",
        "networks": net_names,
        "number_of_episodes": data.get("number_of_episodes"),
        "number_of_seasons": data.get("number_of_seasons"),
        "origin_country": ", ".join(data.get("origin_country", [])) or "",
        "original_language": data.get("original_language"),
        "original_name": data.get("original_name"),
        "overview": data.get("overview"),
        "popularity": data.get("popularity"),
        "poster_path": data.get("poster_path"),
        "production_companies": list_to_str(data.get("production_companies")),
        "production_countries": list_to_str(data.get("production_countries")),
        "seasons": "; ".join(
            f"S{s['season_number']}: {s.get('name')} ({s.get('episode_count')} eps)"
            for s in (data.get("seasons") or []) if s.get("season_number") is not None
        ),
        "spoken_languages": list_to_str(data.get("spoken_languages")),
        "status": data.get("status"),
        "tagline": data.get("tagline"),
        "type_detail": data.get("type"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
    }

    # 리뷰
    rv_items = data.get("reviews", {}).get("results", [])
    series["review"] = " || ".join([
        f"{r.get('author', '')}({r.get('author_details', {}).get('rating', '')}): {(r.get('content') or '').replace(chr(10), ' ')[:200]}"
        for r in rv_items[:5]
    ])

    # 키워드
    kw_items = data.get("keywords", {}).get("results", [])
    series["keyword"] = ", ".join([k["name"] for k in kw_items[:20] if k.get("name")])

    # Cast & Crew
    credits = data.get("aggregate_credits", {})
    cast = credits.get("cast") or []
    crew = credits.get("crew") or []

    series["top_cast"] = ", ".join([c["name"] for c in cast[:4] if c.get("name")])

    dirs, wrs = set(), set()
    for c in crew:
        nm = c.get("name")
        if nm:
            for job in (c.get("jobs") or []):
                jn = job.get("job", "")
                if "Director" in jn:
                    dirs.add(nm)
                if jn in ["Writer", "Screenplay", "Story"]:
                    wrs.add(nm)

    series["directors"] = ", ".join(sorted(dirs)[:10])
    series["writers"] = ", ".join(sorted(wrs)[:10])

    # 🔥 providers + external_ids + seasons 병렬 수집
    seasons_meta = [s for s in (data.get("seasons") or []) if s.get("season_number") is not None]
    
    tasks = [
        fetch_providers(session, sid),      # 0: (flatrate, rent, buy)
        fetch_external_ids(session, sid),   # 1: imdb_id
    ]
    tasks.extend([
        fetch_single_season(session, sid, data.get("name", ""), net_names, net_ids, s)
        for s in seasons_meta
    ])
    
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # Providers 처리 (flatrate, rent, buy)
    if isinstance(results[0], tuple) and len(results[0]) == 3:
        series["providers_flatrate"] = results[0][0]
        series["providers_rent"] = results[0][1]
        series["providers_buy"] = results[0][2]
    else:
        series["providers_flatrate"] = "{}"
        series["providers_rent"] = "{}"
        series["providers_buy"] = "{}"
    
    # IMDB ID 처리
    series["imdb_id"] = results[1] if isinstance(results[1], str) else ""
    
    # Seasons 처리
    season_records = [r for r in results[2:] if r and not isinstance(r, Exception)]

    return series, season_records

# ==========================================================
# 비동기 CSV 쓰기
# ==========================================================
executor = ThreadPoolExecutor(max_workers=2)

async def save_to_csv_async(df, path, mode, header):
    loop = asyncio.get_event_loop()
    await loop.run_in_executor(
        executor,
        lambda: df.to_csv(path, mode=mode, header=header, index=False, encoding="utf-8-sig")
    )

# ==========================================================
# 결측치 보완
# ==========================================================
def fill_series_gaps(series_df, seasons_df):
    print("\n📊 시즌 기반 결측치 보완")

    rtmap = seasons_df.groupby("series_id")["avg_episode_runtime"].mean().round()
    mask = series_df["episode_run_time"].astype(str).str.strip().isin(["", "nan"])
    series_df.loc[mask, "episode_run_time"] = (
        series_df.loc[mask, "id"].map(rtmap).fillna(0).astype(int).astype(str)
    )

    epmap = seasons_df.groupby("series_id")["total_episodes"].sum()
    mask2 = series_df["number_of_episodes"].isna()
    series_df.loc[mask2, "number_of_episodes"] = series_df.loc[mask2, "id"].map(epmap)

    seasmap = seasons_df.groupby("series_id")["season_number"].nunique()
    mask3 = series_df["number_of_seasons"].isna()
    series_df.loc[mask3, "number_of_seasons"] = series_df.loc[mask3, "id"].map(seasmap)

    lastair = seasons_df.sort_values("air_date").groupby("series_id")["air_date"].last()
    ladmask = series_df["last_air_date"].astype(str).str.strip().isin(["", "nan", "NaT"])
    series_df.loc[ladmask, "last_air_date"] = series_df.loc[ladmask, "id"].map(lastair)

    print("✅ 완료\n")
    return series_df

# ==========================================================
# MAIN
# ==========================================================
async def main():
    print("=" * 90)
    print("🚀 TMDB HYPER-OPTIMIZED ASYNC COLLECTOR")
    print("✨ 최종 개선: providers(flatrate/rent/buy 분리) + IMDB ID 추가")
    print("=" * 90)
    stats["start_time"] = datetime.now()
    t0 = datetime.now()

    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    # 🔥 최적 설정
    connector = aiohttp.TCPConnector(
        limit=150,
        limit_per_host=75,
        ttl_dns_cache=600,
        force_close=False,
        enable_cleanup_closed=True
    )

    async with aiohttp.ClientSession(connector=connector, timeout=TIMEOUT) as session:
        # 1) ID 수집
        print("\n📌 1단계: ID 수집")
        ids = sorted(list(await collect_ids_in_range(session, START_DATE, END_DATE)))
        print(f"✨ 총 ID: {len(ids):,}개")

        # 2) 상세 수집
        print("\n📌 2단계: 상세 + 시즌 + IMDB ID + Providers 수집")
        batch_size = 1500
        sem = asyncio.Semaphore(30)  # 🔥 안정적인 동시성

        async def fetch_one(i):
            async with sem:
                return await fetch_tv_details_and_seasons(session, i)

        processed = 0
        last_print = time.time()
        
        for start_idx in range(0, len(ids), batch_size):
            batch_start = time.time()
            part = ids[start_idx:start_idx+batch_size]
            
            results = await asyncio.gather(*[fetch_one(x) for x in part], return_exceptions=True)
            
            bs, bse = [], []
            for r in results:
                if isinstance(r, tuple) and r[0]:
                    s, ss = r
                    bs.append(s)
                    bse.extend(ss)
                processed += 1

            # CSV 쓰기
            if bs:
                await save_to_csv_async(pd.DataFrame(bs), SERIES_TEMP, "a", not SERIES_TEMP.exists())
            if bse:
                await save_to_csv_async(pd.DataFrame(bse), SEASONS_TEMP, "a", not SEASONS_TEMP.exists())

            # 진행 상황 (5초마다)
            now = time.time()
            if now - last_print >= 5:
                elapsed = (datetime.now() - t0).total_seconds()
                rate = processed / elapsed if elapsed > 0 else 0
                eta = (len(ids) - processed) / rate / 60 if rate > 0 else 0
                batch_time = now - batch_start
                print(f"⏱ {processed:,}/{len(ids):,} ({processed/len(ids)*100:.1f}%) | "
                      f"{rate:.1f}/s | ETA: {eta:.0f}분 | Batch: {batch_time:.1f}s | "
                      f"요청: {stats['requests']:,} | 에러: {stats['errors']} | 재시도: {stats['retries']}")
                last_print = now

    # 3) 최종 처리
    print("\n📌 3단계: 최종 처리")
    df_series = pd.read_csv(SERIES_TEMP)
    df_seasons = pd.read_csv(SEASONS_TEMP) if SEASONS_TEMP.exists() else pd.DataFrame()

    SERIES_COLS = [
        "id","title","type","adult","backdrop_path","created_by","episode_run_time",
        "first_air_date","genres","genre_ids","homepage","in_production","languages",
        "last_air_date","last_episode_to_air_id","last_episode_to_air_name",
        "last_episode_to_air_overview","last_episode_to_air_vote_average",
        "last_episode_to_air_vote_count","last_episode_to_air_air_date",
        "last_episode_to_air_episode_number","last_episode_to_air_production_code",
        "last_episode_to_air_runtime","last_episode_to_air_season_number",
        "last_episode_to_air_show_id","last_episode_to_air_still_path",
        "next_episode_to_air","networks","number_of_episodes","number_of_seasons",
        "origin_country","original_language","original_name","overview","popularity",
        "poster_path","production_companies","production_countries","seasons",
        "spoken_languages","status","tagline","type_detail","vote_average",
        "vote_count","review","keyword","top_cast","directors","writers",
        "providers_flatrate","providers_rent","providers_buy","imdb_id"  # ✨ 변경된 컬럼들
    ]

    SEASON_COLS = [
        "_id","season_id","series_id","series_name","season_number","name","air_date",
        "overview","vote_average","vote_count","network_names","network_ids",
        "total_episodes","avg_episode_runtime","poster_path"
    ]

    df_series = df_series[SERIES_COLS].drop_duplicates(subset=["id"])
    if not df_seasons.empty:
        df_seasons = df_seasons[SEASON_COLS].drop_duplicates(subset=["series_id","season_number"])
        df_series = fill_series_gaps(df_series, df_seasons)

    df_series.to_csv(SERIES_CSV, index=False, encoding="utf-8-sig")
    df_seasons.to_csv(SEASONS_CSV, index=False, encoding="utf-8-sig")

    try:
        df_series.to_parquet(SERIES_PARQ, index=False)
        df_seasons.to_parquet(SEASONS_PARQ, index=False)
    except:
        pass

    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    elapsed = (datetime.now() - t0).total_seconds()/60

    print("="*90)
    print("🎉 수집 완료!")
    print("="*90)
    print(f"📌 Series: {len(df_series):,}개 | Seasons: {len(df_seasons):,}개")
    print(f"📌 API 요청: {stats['requests']:,}회 | 에러: {stats['errors']} | 재시도: {stats['retries']}")
    print(f"📌 평균 속도: {len(df_series)/elapsed:.1f} series/분")
    print(f"⏱ 총 시간: {elapsed:.1f}분 ({elapsed/60:.1f}시간)")
    print("="*90)
    
    # 샘플 데이터 출력
    print("\n📊 샘플 Providers 데이터:")
    if not df_series.empty and 'providers_flatrate' in df_series.columns:
        sample = df_series[df_series['providers_flatrate'] != '{}'].head(1)
        if not sample.empty:
            print(f"Title: {sample.iloc[0]['title']}")
            print(f"Flatrate: {sample.iloc[0]['providers_flatrate'][:200]}...")
            print(f"IMDB ID: {sample.iloc[0]['imdb_id']}")
    
    executor.shutdown(wait=False)

if __name__ == "__main__":
    asyncio.run(main())